But it's so boring. Also it's not full automated way.

Unfortunately, pd.read_html and requests + beautifulsoup do not allow parsing info.
Therefore, I decided to use API.

First and foremost, I registered in https://etherscan.io/ to get API key.

FREE API PLAN allows doing 5 calls per second. It's appropriate number for our goals.

API-Key is kept in .env file.But it's so boring. Also it's not full automated way.

Unfortunately, pd.read_html and requests + beautifulsoup do not allow parsing info.
Therefore, I decided to use API.

First and foremost, I registered in https://etherscan.io/ to get API key.

FREE API PLAN allows doing 5 calls per second. It's appropriate number for our goals.

API-Key is kept in .env file.

In [7]:
import requests
from bs4 import BeautifulSoup
from application.load_transaction_data import load_data
df = load_data()
token_transactions_df = df[df[['Value_IN(ETH)', 'Value_OUT(ETH)']].sum(axis=1) == 0]
ether_transactions_df = df[df[['Value_IN(ETH)', 'Value_OUT(ETH)']].sum(axis=1) > 0]

In [5]:
token_transactions_df = df[df[['Value_IN(ETH)', 'Value_OUT(ETH)']].sum(axis=1) == 0]

In [13]:
import pandas as pd
import re
from tqdm import tqdm

def extract_token_info(token_url):
    result_dict = {}
    try:
        result_dict['from'] =  token_url.find("span", class_='hash-tag text-truncate hash-tag-custom-from tooltip-address').text
    except Exception as e:
        result_dict['from'] = e
    try:
        result_dict['to'] = token_url.find("span", class_='hash-tag text-truncate hash-tag-custom-to tooltip-address').text
    except Exception as e:
        result_dict['to'] = e
    try:
        token1_amount = token_url.find("span", 'data-toggle'=='tooltip').text
        result_dict['amount'] = float(token1_amount.replace(",", "_"))
    except Exception as e:
        result_dict['amount'] = e
    try:
        # token1_usd = token_url.find("button", class_='u-label u-label--value u-label--primary rounded mb-n1 ml-1').text
        token1_usd = re.findall(r"\(\$.*?\)", str(token_url))[0]
        token1_usd = token1_usd.strip("()").replace(" (", "").replace("$", "")
        result_dict['usd'] = float(token1_usd.replace(",", "_"))
    except Exception as e:
        result_dict['usd'] = e
    try:
        result_dict['token_hash'] = token_url.find("a").get("href").split("?")[0].split("/")[2]
    except Exception as e:
        result_dict['token_hash'] = e
    return result_dict

def extract_tokens_info(transaction_hash):
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
    req = requests.get(f"https://etherscan.io/tx/{transaction_hash}",
                   headers=headers)
    content = req.content
    soup = BeautifulSoup(content)

    tokens_df = []
    bad_tokens = []
    errors = []
    all_tokens = soup.find_all("li", class_='media align-items-baseline mb-2')
    for index, tokens in enumerate(all_tokens):
        try:
            tokens_df.append(
                pd.DataFrame.from_dict(extract_token_info(all_tokens[index]), orient='index').T
            )
        except Exception as e:
            bad_tokens.append(index)
            errors.append(e)
    try:

        tokens_df = pd.concat(tokens_df)
    except ValueError:
        tokens_df = pd.DataFrame(data=[transaction_hash], columns=['Txhash'])
    tokens_df['Txhash'] = transaction_hash
    return tokens_df

all_tokens_df = []
for tx_hash in tqdm(token_transactions_df['Txhash'].unique().tolist()):
    all_tokens_df.append(extract_tokens_info(tx_hash))
all_tokens_df = pd.concat(all_tokens_df)
all_tokens_df.to_pickle("all_tokends_df")

100%|██████████| 6739/6739 [1:08:43<00:00,  1.63it/s]  


In [31]:
all_tokens_df['amount'] = pd.to_numeric(all_tokens_df['amount'], errors='coerce')
all_tokens_df['usd'] = pd.to_numeric(all_tokens_df['usd'], errors='coerce')

In [87]:
from fake_useragent import UserAgent
ua = UserAgent(use_cache_server=False)
proxies = {"https": "188.170.233.113:3128"}
transaction_hash = "https://etherscan.io/tx/0x3f47026093d13668760dc35418cb4983b374e9ebdc5af2cca2d93559b83dd10f"
req = requests.get(f"https://etherscan.io/tx/{transaction_hash}",
                   headers={"User-Agent":ua.random},
                   # proxies=proxies
                   )
content = req.content
soup = BeautifulSoup(content)
all_tokens = soup.find_all("li", class_='media align-items-baseline mb-2')
token_url = all_tokens[0]
token1_amount = token_url.find("span", 'data-toggle'=='tooltip').text
amount = float(token1_amount.replace(",", "_"))

IndexError: list index out of range

In [82]:
token_transactions_df = token_transactions_df.merge(all_tokens_df,
                            how='left', on='Txhash')

In [88]:
soup

<!DOCTYPE html>
<html lang="en">
<head><title>Etherscan Error Page</title>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="The Ethereum BlockChain Explorer, API and Analytics Platform" name="Description"/>
<meta content="etherscan.io" name="author"/>
<meta content="ethereum, explorer, ether, search, blockchain, crypto, currency" name="keywords"/>
<meta content="telephone=no" name="format-detection"/>
<link href="/images/favicon2.ico" rel="shortcut icon"/>
<link href="/assets/vendor/font-awesome/css/fontawesome-all.min.css?v=0.11.4.0" rel="stylesheet"/>
<link href="/assets/css/theme.min.css?v=20.2.4.0" rel="stylesheet"/>
<script async="" src="/cdn-cgi/challenge-platform/h/b/scripts/invisible.js?ts=1651042800"></script>
</head><body>
<header class="u-header" id="header">
<div class="u-header__section">
<div class="container" id="logoAndNav">
<nav class="js-mega-menu navbar navbar-expand-md u-header__navbar u-header__navbar--no-space 

In [85]:
token_transactions_df

,Txhash,Blockno,DateTime,From,To,Value_IN(ETH),Value_OUT(ETH),TxnFee(USD),Historical $Price/Eth,Status,ErrCode,Method,from,to,amount,usd,token_hash
0,0x2394fdb6228d761f899156ebbc06554d0701c5952500...,14644335,2022-04-24 00:40:23,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0xf403c135812408bfbe8713b5a23a04b3d48aae31,0.0,0.0,35.279306,2922.37,NaN,NaN,Deposit All,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,Convex Finance: Voter Proxy,NaN,NaN,0x5282a4ef67d9c33135340fb3289cc1711c13638c
1,0x2394fdb6228d761f899156ebbc06554d0701c5952500...,14644335,2022-04-24 00:40:23,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0xf403c135812408bfbe8713b5a23a04b3d48aae31,0.0,0.0,35.279306,2922.37,NaN,NaN,Deposit All,Convex Finance: Voter Proxy,Curve.fi: IronBank Liquidity Gauge,NaN,NaN,0x5282a4ef67d9c33135340fb3289cc1711c13638c
2,0x2394fdb6228d761f899156ebbc06554d0701c5952500...,14644335,2022-04-24 00:40:23,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0xf403c135812408bfbe8713b5a23a04b3d48aae31,0.0,0.0,35.279306,2922.37,NaN,NaN,Deposit All,Null Address: 0x000…000,Convex Finance: Voter Proxy,NaN,NaN,0xf5194c3325202f456c95c1cf0ca36f8475c1949f
3,0x2394fdb6228d761f899156ebbc06554d0701c5952500...,14644335,2022-04-24 00:40:23,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0xf403c135812408bfbe8713b5a23a04b3d48aae31,0.0,0.0,35.279306,2922.37,NaN,NaN,Deposit All,Null Address: 0x000…000,Convex Finance: Booster,NaN,NaN,0x912ec00eaebf3820a9b0ac7a5e15f381a1c91f22
4,0x2394fdb6228d761f899156ebbc06554d0701c5952500...,14644335,2022-04-24 00:40:23,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0xf403c135812408bfbe8713b5a23a04b3d48aae31,0.0,0.0,35.279306,2922.37,NaN,NaN,Deposit All,Convex Finance: Booster,0x3e03fff82f77073cc590b656d42fceb12e4910a8,NaN,NaN,0x912ec00eaebf3820a9b0ac7a5e15f381a1c91f22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18057,0xb550117ab3e407bc203ad24ae42a64a16971e833f9ee...,10601645,2020-08-05 19:31:01,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0.0,0.0,4.363306,401.12,NaN,NaN,Approve,NaN,NaN,NaN,NaN,NaN
18058,0x720f6a4d8999ae940cb2df3a633034ea9ca1078bef97...,10601597,2020-08-05 19:19:02,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0x4678f0a6958e4d2bc4f1baf7bc52e8f3564f3fe4,0.0,0.0,67.162914,401.12,NaN,NaN,Build,NaN,NaN,NaN,NaN,NaN
18059,0x63f876ae7c626619a740f26d135dc187f2c1dd24d42f...,10601303,2020-08-05 18:18:49,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0x32666b64e9fd0f44916e1378efb2cfa3b3b96e80,0.0,0.0,12.164124,401.12,NaN,NaN,Mint,Null Address: 0x000…000,Ren: RenBridge,99.899301,1162749.46,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d
18060,0x63f876ae7c626619a740f26d135dc187f2c1dd24d42f...,10601303,2020-08-05 18:18:49,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0x32666b64e9fd0f44916e1378efb2cfa3b3b96e80,0.0,0.0,12.164124,401.12,NaN,NaN,Mint,Null Address: 0x000…000,RenVM: Darknodes Fee renBTC,0.099999,1163.91,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d
